In [1]:
import os
import torch
import random
import torch.nn as nn
import torch.backends.cudnn as cudnn
import numpy as np
from models import build_model
from PIL import Image
from IPython.display import Image as ipython_image
from diffusers.utils import load_image, export_to_video, export_to_gif

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


In [2]:
import torch
torch.manual_seed(0)


In [3]:
from conversation import default_conversation, DEFAULT_IMAGE_TOKEN, DEFAULT_VIDEO_TOKEN, VIDEO_TOKEN_INDEX, IMAGE_TOKEN_INDEX
DEFAULT_VISUAL_TOKEN = DEFAULT_VIDEO_TOKEN
VISUAL_TOKEN_INDEX = VIDEO_TOKEN_INDEX

In [4]:
from datasets import load_dataset
dataset30 = load_dataset("lmms-lab/LLaVA-Video-178K", '0_30_s_academic_v0_1')

In [5]:
import pandas as pd
from datasets import Dataset
open_ended = pd.DataFrame(dataset30['open_ended'])
caption = pd.DataFrame(dataset30['caption'])
overall_df = pd.concat([open_ended, caption])
dataset = Dataset.from_pandas(overall_df)
shuffled_dataset = dataset.shuffle(seed=42)

In [6]:
from models.modeling_video_lavit_hf import VideoLaVITLlamaForCausalLM
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor


In [7]:
import transformers
img_size=224
model_dtype="bf16",
apply_lemmatizer=True
use_xformers=False
max_frames=24
max_clips=8
motion_vocab_size=1026
visual_vocab_size = 16384

device_map={"": 0}

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'
config = transformers.AutoConfig.from_pretrained(model_path)
config.use_xformers = use_xformers

# For inference, we should use the left padding
config.tokenizer_padding_side = 'left'
config.use_cache = True

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token
print (tokenizer.pad_token, tokenizer.unk_token)

model = VideoLaVITLlamaForCausalLM.from_pretrained(model_path, config=config, device_map=device_map,
                torch_dtype=torch.bfloat16 if model_dtype=="bf16" else torch.float16,)

for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in model.model.visual_tokenizer.named_parameters():
    
    param.requires_grad = True

for name, param in model.model.motion_tokenizer.named_parameters():
    param.requires_grad = True

model.train()

visual_vocab_size = visual_vocab_size


<unk> <unk>
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.cluster_size']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor
video_processor = LaVITEvalVideoProcessor(image_size=224, num_frames=24, fps=6, max_clips=8,)

## helping function

In [9]:
from conversation import default_conversation, DEFAULT_IMAGE_TOKEN, DEFAULT_VIDEO_TOKEN, VIDEO_TOKEN_INDEX, IMAGE_TOKEN_INDEX, IGNORE_INDEX, MODAL_INDEX_MAP
from typing import Dict, Optional, Sequence, List
import copy
import transformers

In [10]:
def tokenizer_multimodal_token(prompt, tokenizer, image_token_index=VIDEO_TOKEN_INDEX, image_token=DEFAULT_IMAGE_TOKEN, return_tensors=None):
    """Tokenize text and multimodal tag to input_ids.

    Args:
        prompt (str): Text prompt (w/ multimodal tag), e.g., '<video>\nDescribe the video.'
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer object.
        multimodal_token (int): Token index corresponding to the multimodal tag.
    """
    prompt_chunks = [tokenizer(chunk, add_special_tokens=False).input_ids for idx, chunk in enumerate(prompt.split(image_token))]

    input_ids = []
    for i in range(1, 2 * len(prompt_chunks)):
        if i % 2 == 1:
            input_ids.extend(prompt_chunks[i // 2])
        else:
            input_ids.append(image_token_index)

    if return_tensors is not None:
        if return_tensors == 'pt':
            return torch.tensor(input_ids, dtype=torch.long)
        raise ValueError(f'Unsupported tensor type: {return_tensors}')
    return input_ids

In [11]:
def tokenizer_image_token(prompt, tokenizer, image_token_index=VIDEO_TOKEN_INDEX, image_token=DEFAULT_IMAGE_TOKEN, return_tensors=None):
    prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split(image_token)]

    def insert_separator(X, sep):
        return [ele for sublist in zip(X, [sep]*len(X)) for ele in sublist][:-1]

    input_ids = []
    offset = 0
    if len(prompt_chunks) > 0 and len(prompt_chunks[0]) > 0 and prompt_chunks[0][0] == tokenizer.bos_token_id:
        offset = 1
        input_ids.append(prompt_chunks[0][0])

    for x in insert_separator(prompt_chunks, [image_token_index] * (offset + 1)):
        input_ids.extend(x[offset:])

    if return_tensors is not None:
        if return_tensors == 'pt':
            return torch.tensor(input_ids, dtype=torch.long)
        raise ValueError(f'Unsupported tensor type: {return_tensors}')
    return input_ids

In [12]:
def apply_chat_template(source, roles):
    conv = default_conversation.copy()
    for sentence in source:
        conv.append_message(sentence['role'], sentence['content'])
    return conv.get_prompt()

In [13]:
def preprocess(
    sources: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
    modal_token: str = None,
) -> Dict:
    roles = {"human": "USER", "gpt": "ASSISTANT"}

    # Apply prompt templates
    conversations = []
    input_ids = []
    targets = []
    #print (len(sources), sources)
    for i, source in enumerate(sources):
        #print ("sourse", source, "\n")
        #print ("source", source)
        #print (roles[source[0]["from"]])
        if roles[source[0]["from"]] != "USER":
            # Skip the first one if it is not from human
            source = source[1:]
        #message = [{'role': roles[sentence['from']], 'content': sentence['value']} for sentence in source]
        conversation = default_conversation.copy()#tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False)
        conv = default_conversation.copy()
        for sentence in source:
            conv.append_message(roles[sentence['from']], sentence['value'])
        #print (conv.get_prompt())
        input_ids.append(tokenizer_image_token(conv.get_prompt(), tokenizer, return_tensors='pt'))
        targets.append(copy.deepcopy(input_ids[-1]))

        #print ("\n input_ids", input_ids)
        #print ("\n targets", targets)

        

        assert len(source) % 2 == 0, f"Invalid conversation length {len(source)}."

        cur = 0
        message = []
        for idx, sentence in enumerate(source):
            if idx % 2 == 1:
                tmp_message = [
                    {'role': roles[source[idx-1]['from']], 'content': source[idx-1]['value']}, 
                    {'role': roles[sentence['from']], 'content': sentence['value']}
                ]

                #print ("message + tmp_message[:1]", message + tmp_message[:1])
                #print (message + tmp_message)
                instruction = apply_chat_template(message + tmp_message[:1], roles)#tokenizer.apply_chat_template(message + tmp_message[:1], tokenize=False, add_generation_prompt=True)
                conversation = apply_chat_template(message + tmp_message, roles)

    
                #print ("\n\n\n",idx)
                #print ("\n\n",idx)
                #print ("instruction", instruction)
                #print ("conversation", conversation)
                
                instruction_len = len(tokenizer_image_token(instruction, tokenizer, return_tensors='pt'))
                conversation_len = len(tokenizer_image_token(conversation, tokenizer, return_tensors='pt'))

                targets[-1][cur:instruction_len+4] = IGNORE_INDEX

                cur = conversation_len
                message += tmp_message

    return dict(input_ids=input_ids, labels=targets)

In [14]:
def collater(instances):
        #print ("instances", instances)
        #print ("instances0", instances[0])
        input_ids = instances[0]["input_ids"]
        #print ("input_ids1", input_ids)
        input_ids = [torch.tensor(elem) for elem in input_ids]
        #print ("input_ids2", input_ids)
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids,
            batch_first=True,
            padding_value=tokenizer.pad_token_id)
        labels = instances[0]["labels"]
        labels = [torch.tensor(elem) for elem in labels]
        labels = torch.nn.utils.rnn.pad_sequence(labels,
                                                 batch_first=True,
                                                 padding_value=IGNORE_INDEX)
        batch = dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(tokenizer.pad_token_id),
            video_pathes = instances[0]["video"],
            data_sources = instances[0]["data_source"]
        )

        return batch

In [15]:
a = dataset30['open_ended'].map(lambda x: preprocess([x['conversations']], tokenizer),remove_columns=['id', 'conversations'])

## pl training

In [16]:
# this is smoothed version of nll loss from pytorch trainer 
# TO DO: try to replace
# https://github.com/huggingface/transformers/blob/76da6ca0349d4beb176ec6bc99d218a47980e82c/src/transformers/trainer_pt_utils.py#L554
def smooth_loss(logits, labels, ignore_index):
    log_probs = -nn.functional.log_softmax(logits, dim=-1)
    if labels.dim() == log_probs.dim() - 1:
            labels = labels.unsqueeze(-1)

    padding_mask = labels.eq(self.ignore_index)
    # In case the ignore_index is -100, the gather will fail, so we replace labels by 0. The padding_mask
    # will ignore them in any case.
    labels = torch.clamp(labels, min=0)
    nll_loss = log_probs.gather(dim=-1, index=labels)
    # works for fp16 input tensor too, by internally upcasting it to fp32
    smoothed_loss = log_probs.sum(dim=-1, keepdim=True, dtype=torch.float32)

    nll_loss.masked_fill_(padding_mask, 0.0)
    smoothed_loss.masked_fill_(padding_mask, 0.0)

    # Take the mean over the label dimensions, then divide by the number of active elements (i.e. not-padded):
    num_active_elements = padding_mask.numel() - padding_mask.long().sum()
    nll_loss = nll_loss.sum() / num_active_elements
    smoothed_loss = smoothed_loss.sum() / (num_active_elements * log_probs.shape[-1])
    return (1 - self.epsilon) * nll_loss + self.epsilon * smoothed_loss

In [17]:
import pytorch_lightning as pl
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from models.video_lavit_for_understanding import tokenizer_image_token
from torch.utils.data import DataLoader, Dataset

In [18]:
VIDEO_PATH = "/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/"
import os
cwd = os.getcwd()

In [19]:
VISUAL_TOKEN_INDEX

-201

In [20]:
input_ids = torch.tensor(a[1]['labels'])
input_ids.shape

torch.Size([1, 259])

In [21]:
input_ids[:,0:35]

tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [22]:
torch.where(input_ids == VISUAL_TOKEN_INDEX)

(tensor([], dtype=torch.int64), tensor([], dtype=torch.int64))

In [23]:

"""
    Training parameters are from https://arxiv.org/html/2402.03161v3#bib.bib48 sec. A.3
"""

class VideoLaVIT_pl(pl.LightningModule):
    def __init__(self, video_processor, model, train_dataset, tokenizer, collate_function):
        super(VideoLaVIT_pl, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        #self.valid_dataset = valid_dataset
        self.video_processor = video_processor
        #self.n_embeddings = model.model.embed_tokens.weight.shape[0]
        self.loss = nn.CrossEntropyLoss(reduction="none", ignore_index=IGNORE_INDEX)
        #self.train_dataset = train_dataset
        #self.validation_dataset = dataset_valid
        self.collate_function = collate_function
        self.n_iters = 10000# len(self.train_dataloader())
        self.n_warmup = 1000
        self.grad_clip = 1.0
        self.weight_decay = 0.001
        self.grad_accum = 256
        self.lr = 0.0001
        self.beta1 = 0.9
        self.beta2 = 0.99
        self.collater = collate_function
        self.losses = []
        self.n_iters = len(self.train_dataloader())
        # self.automatic_optimization = False

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=1, collate_fn=self.collater)

    def process_video(self, video_inputs, num_beams=1):
        if not isinstance(video_inputs, list):
            if not isinstance(video_inputs, str):
                raise ValueError("The video_inputs should be Tensors or Str (video path)")
            video_inputs = [video_inputs]
    
        if isinstance(video_inputs[0], list):
            assert isinstance(video_inputs[0][0], torch.Tensor)
            video_inputs_list = []
            for video_input in video_inputs:
                for _ in range(num_beams):
                    video_inputs_list.append(((video_input[0], video_input[1]), 'video'))
            return video_inputs_list
            
        if isinstance(video_inputs[0], str):
            video_inputs_list = []
            for video_path in video_inputs:
                visual_inputs, motion_inputs = video_processor(video_path)
                for _ in range(num_beams):
                    video_inputs_list.append(((visual_inputs, motion_inputs), 'video'))
    
            return video_inputs_list
        
        
    def configure_optimizers(self):
        
        optimizer = AdamW(list(self.model.model.visual_tokenizer.parameters()) + list(self.model.model.motion_tokenizer.parameters()), lr=self.lr, betas =(self.beta1, self.beta2), weight_decay = self.weight_decay)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=self.n_warmup, num_training_steps=self.n_iters // self.grad_accum)
        return {'optimizer': optimizer, 'lr_scheduler': {
            'scheduler': scheduler,
            'interval': 'step',
            'frequency': 1
            
        }}
    

    def on_train_epoch_end(self):
        print ("epoch end")
        torch.save(self.model.model.visual_tokenizer.state_dict(), cwd + f"/custom_ckpts/visual_tokenizer/" + "model.pth")
        torch.save(self.model.model.motion_tokenizer.state_dict(), cwd+ f"/custom_ckpts/motion_tokenizer/" + "model.pth")

        
    def training_step(self, batch, batch_idx):
        labels = batch['labels']
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        video_path = batch['video_pathes']
        folder = batch['data_sources']
        full_path = VIDEO_PATH + folder + "/" + video_path
        visual_inputs = self.process_video(full_path)

        #print ("input_ids shape", input_ids.shape)     

        #print ("\ninput_ids", input_ids)     
        
        image_token_indices = [-1] + torch.where(input_ids == VISUAL_TOKEN_INDEX)[1].tolist() + [input_ids.shape[1]]
        #print ("\nimage_token_indices!", image_token_indices)
        cur_labels_noim = []
        for i in range(len(image_token_indices) - 1):
            cur_labels_noim.append(labels[:, image_token_indices[i]+1:image_token_indices[i+1]])


        #print ("\ncur_labels_noim", cur_labels_noim, "\n")

        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            logits = self.model(input_ids = input_ids, attention_mask = attention_mask, images = visual_inputs).logits


        video_seq_len = logits.shape[1] - labels.shape[1] +1

        new_labels = []
        new_labels.append(cur_labels_noim[0])
        new_labels.append(torch.full((1, video_seq_len), IGNORE_INDEX, device=labels.device, dtype=labels.dtype))
        new_labels.append(cur_labels_noim[1])


        new_labels = torch.cat(new_labels,1)

        # VideoLLama2 is trained using pytorch trainer. In this Trainer, for casual LM label shifting is applied 
        #logits = logits[..., :-1, :].contiguous()
        #new_labels = new_labels[..., 1:].contiguous()
        
        loss = self.loss(logits.view(-1, logits.shape[2]), new_labels.view(-1)).mean()
        self.losses.append(loss.cpu().detach().numpy())           
        self.log("my_loss", np.mean(self.losses[-20:]), on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        #print (batch[0]['question'])
        labels = batch['labels']
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        video_path = batch['video_pathes']
        folder = batch["data_source"]
        full_path = VIDEO_PATH + folder + video_path
        print ("FULL path", full_path)
        visual_inputs = self.process_video(full_path)

        print ("input_ids.shape", input_ids.shape)
        

        with torch.no_grad():
            logits = self.model(input_ids = input_ids, attention_mask = attention_mask, images = visual_inputs).logits

        print ("logits.shape", logits.shape)
        print ("labels.shape", labels.shape)

        logits = logits[:,:labels.shape[1],:]

        logits = logits[..., :-1, :].contiguous()
        labels = labels[..., 1:].contiguous()
        
        loss = self.loss(logits.view(-1, logits.shape[2]), new_labels.view(-1)).mean()
        loss.requires_grad = True

        
        self.log("val_loss", loss, on_step=True, batch_size=1, on_epoch=True, prog_bar=True)
        

In [24]:
module = VideoLaVIT_pl(video_processor, model, a, tokenizer, collater)


In [25]:
# module.model.model.visual_tokenizer <-- what we are saving

In [26]:
from pytorch_lightning.loggers import CSVLogger
logger = CSVLogger("ckpts", name="validation")

In [ ]:
trainer = pl.Trainer(devices=1, max_epochs=1, max_steps=1000, log_every_n_steps=100, logger =logger)
trainer.fit(module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprec

Training: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_siz

In [32]:
tokenizer.decode([2860, 16254,   292,
          411,   278, 11757, 29884,   398, 27372, 29892,   278,  2022, 15028,
          701, 29892,  6576, 11480,   278,  2319, 10200,  1230,  3800, 29892,
        15028,   727,   363,   263, 11473,  3256, 29892,   769,  6576,  1283,
          322,   429,  1169,   278,  3515, 29889,   320, 29876, 29871,     2])

' After interacting with the vacuum cleaner, the person stands up, steps onto the small decorative box, stands there for a brief moment, then steps off and exits the frame. \\n </s>'

In [29]:
model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.cluster_size']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


# reg

In [30]:
video_path = '/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4'
prompt = "Where does the video take place?"
answer = "In the kitchen"
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

text inputs ['<video>\nWhere does the video take place?']
text input <video>
Where does the video take place?
Conversation(system="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", roles=('USER', 'ASSISTANT'), messages=[], offset=0, sep_style=<SeparatorStyle.TWO: 2>, sep=' ', sep2=' \\n </s>', version='v1', skip_next=False)
[['USER', '<video>\nWhere does the video take place?']]
[['USER', '<video>\nWhere does the video take place?'], ['ASSISTANT', None]]
prompt A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>
Where does the video take place? ASSISTANT:
prompts ["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>\nWhere does the video take place? A

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


The video takes place in a garage where we see a woman wearing black and purple shoes, black clothes, and a ponytail cleaning a carpet with a vacuum cleaner. There is also another person standing on the floor next to the woman.


## tuned visual tokenizer

In [31]:
dict_ = torch.load("/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/custom_ckpts/visual_tokenizer/model.pth")

In [32]:
torch.equal(dict_['encoder.blocks.10.attn.proj.weight'], runner.model.model.visual_tokenizer.encoder.blocks[10].attn.proj.weight)

True

In [37]:
runner.model.model.visual_tokenizer.load_state_dict(dict_)

<All keys matched successfully>

In [34]:
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

text inputs ['<video>\nWhere does the video take place?']
text input <video>
Where does the video take place?
Conversation(system="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", roles=('USER', 'ASSISTANT'), messages=[], offset=0, sep_style=<SeparatorStyle.TWO: 2>, sep=' ', sep2=' \\n </s>', version='v1', skip_next=False)
[['USER', '<video>\nWhere does the video take place?']]
[['USER', '<video>\nWhere does the video take place?'], ['ASSISTANT', None]]
prompt A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>
Where does the video take place? ASSISTANT:
prompts ["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>\nWhere does the video take place? A

# tuned visual tokenizer + motion_tokenizer

In [35]:
runner.model.model.motion_tokenizer.load_state_dict(torch.load("/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/custom_ckpts/motion_tokenizer/model.pth"))

<All keys matched successfully>

In [36]:
video_path = '/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4'
prompt = "Where does the video take place?"
answer = "In the kitchen"
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

text inputs ['<video>\nWhere does the video take place?']
text input <video>
Where does the video take place?
Conversation(system="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", roles=('USER', 'ASSISTANT'), messages=[], offset=0, sep_style=<SeparatorStyle.TWO: 2>, sep=' ', sep2=' \\n </s>', version='v1', skip_next=False)
[['USER', '<video>\nWhere does the video take place?']]
[['USER', '<video>\nWhere does the video take place?'], ['ASSISTANT', None]]
prompt A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>
Where does the video take place? ASSISTANT:
prompts ["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>\nWhere does the video take place? A